# 1.1 Movies Database

Take the movies dataset and turn it into a single `sqlite` database. It should have one table for each csv file in the movies dataset

In [82]:
import sqlite3 
import pandas as pd 

In [ ]:
# Cleaing up the budget column. its filled with nonsense
# I need it clean to get an accurate count for question 1.2.1

In [201]:
df = pd.read_csv("/Users/kalebmckenzie/Documents/GitHub/4-1-sql/movies_metadata.csv")

In [156]:
#cleaing up the budget col so its easier to work with
ran_bud = df[df['budget'] == '/ff9qCepilowshEtG2GYWwzt2bs4.jpg'].index
ran_bud2 = df[df['budget'] == '/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg'].index
ran_bud3 = df[df['budget'] == '/zaSf5OG7V8X8gqFvly88zDdRm46.jpg'].index

df.drop(ran_bud, inplace = True)
df.drop(ran_bud2, inplace = True)
df.drop(ran_bud3, inplace = True)

df['budget'] = df['budget'].astype(int)

In [157]:
df.to_csv('new_movies.csv')

In [83]:
conn = sqlite3.connect('movie.db')
c = conn.cursor()

In [107]:
credits = pd.read_csv('/Users/kalebmckenzie/Documents/GitHub/4-1-sql/credits.csv')
credits.to_sql('credits', conn, if_exists='replace', index = False)

In [108]:
keywords = pd.read_csv('/Users/kalebmckenzie/Documents/GitHub/4-1-sql/keywords.csv')
keywords.to_sql('keywords', conn, if_exists='replace', index = False)

In [109]:
link_s = pd.read_csv('/Users/kalebmckenzie/Documents/GitHub/4-1-sql/links_small.csv')
link_s.to_sql('link_s', conn, if_exists='replace', index = False)

In [110]:
links = pd.read_csv('/Users/kalebmckenzie/Documents/GitHub/4-1-sql/links.csv')
links.to_sql('links', conn, if_exists='replace', index = False)

In [158]:
metadata = pd.read_csv('/Users/kalebmckenzie/Documents/GitHub/4-1-sql/new_movies.csv')
metadata.to_sql('metadata', conn, if_exists='replace', index = False)

In [112]:
rating_s = pd.read_csv('/Users/kalebmckenzie/Documents/GitHub/4-1-sql/ratings_small.csv')
rating_s.to_sql('rating_s', conn, if_exists='replace', index = False)

In [113]:
ratings = pd.read_csv('/Users/kalebmckenzie/Documents/GitHub/4-1-sql/ratings.csv')
ratings.to_sql('ratings', conn, if_exists='replace', index = False)

In [117]:
# A view of the tables

def tables_in_sqlite_db(conn):
    cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [
        v[0] for v in cursor.fetchall()
        if v[0] != "sqlite_sequence"
    ]
    cursor.close()
    return tables

In [118]:
movie_tables = tables_in_sqlite_db(conn)
movie_tables

['credits', 'keywords', 'link_s', 'links', 'rating_s', 'ratings', 'metadata']

In [202]:
#p2 = pd.read_sql('select * from metadata', conn)

# 1.2 Queries

**1.2.1** Use a single query to pull the original title of movies with a budget above $5m

**1.2.2** Use a query to pull the english-language films with the word `war` in their title

**1.2.3** Left join the average ratings from the `ratings` table onto the `movies_metadata` table, so you can have a relation between budget and rating. Hint: use a subquery.

# 1.2.1

In [161]:
q1 = pd.read_sql_query(
"""
SELECT 
    original_title,
    budget

FROM metadata

WHERE budget > 5000000

""", 
    con=conn
)
q1

,original_title,budget
0,Toy Story,30000000
1,Jumanji,65000000
2,Waiting to Exhale,16000000
3,Heat,60000000
4,Sabrina,58000000
...,...,...
5166,Detroit,34000000
5167,சிவாஜி,12000000
5168,Pattaya,5402000
5169,Les Visiteurs: La Révolution,25868826


# 1.2.2

In [180]:
q2 = pd.read_sql_query(
"""
SELECT 
    original_language,
    title

FROM metadata

WHERE original_language = 'en'
    and title LIKE '% war %'
""", 
    con=conn
)
q2

,original_language,title
0,en,The War Room
1,en,The War at Home
2,en,The War of the Worlds
3,en,The War Zone
4,en,The War of the Roses
5,en,At War with the Army
6,en,I Was a Male War Bride
7,en,The War Wagon
8,en,Outfoxed: Rupert Murdoch's War on Journalism
9,en,The War Within


# 1.2.3

In [199]:
q3 = pd.read_sql_query(
"""
SELECT
    budget,
    rating_s.rating

FROM metadata

LEFT JOIN rating_s ON
    metadata.id = rating_s.movieId

ORDER BY rating_s.rating DESC
    
""", con=conn)

q3

,budget,rating
0,60000000,5.0
1,60000000,5.0
2,98000000,5.0
3,98000000,5.0
4,98000000,5.0
...,...,...
87621,0,NaN
87622,0,NaN
87623,0,NaN
87624,0,NaN


In [200]:
#tee = pd.read_csv("/Users/kalebmckenzie/Documents/GitHub/4-1-sql/ratings_small.csv")

# 2. Baseball Database

The [Baseball Database](http://www.seanlahman.com/baseball-archive/statistics/) has an sqlite version. Download it for these exercises.

**2.1** Which player has had the most homeruns?

**2.2** Is there a relation between how many homeruns a player has made in a year and his salary that year? Pull both colums together in a single query



In [98]:
baseball = sqlite3.connect('/Users/kalebmckenzie/Documents/GitHub/4-1-sql/lahmansbaseballdb.sqlite')
b = baseball.cursor()


In [99]:
#Just so I can view the tables in the db

def tables_in_sqlite_dbb(baseball):
    cursor = baseball.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tablesb = [
        v[0] for v in cursor.fetchall()
        if v[0] != "sqlite_sequence"
    ]
    cursor.close()
    return tablesb

In [100]:
baseb_tables = tables_in_sqlite_dbb(baseball)
baseb_tables

['allstarfull',
 'appearances',
 'awardsmanagers',
 'awardsplayers',
 'awardssharemanagers',
 'awardsshareplayers',
 'batting',
 'battingpost',
 'collegeplaying',
 'divisions',
 'fielding',
 'fieldingof',
 'fieldingofsplit',
 'fieldingpost',
 'halloffame',
 'homegames',
 'leagues',
 'managers',
 'managershalf',
 'parks',
 'people',
 'pitching',
 'pitchingpost',
 'salaries',
 'schools',
 'seriespost',
 'teams',
 'teamsfranchises',
 'teamshalf']

# 2.1

In [101]:
b1 = pd.read_sql_query(
"""
SELECT  
    playerID, 
    HR

FROM batting

ORDER BY HR DESC
""", 
    con=baseball
)
b1

#Player Bondsba01 had the most homeruns

,playerID,HR
0,bondsba01,73
1,mcgwima01,70
2,sosasa01,66
3,mcgwima01,65
4,sosasa01,64
...,...,...
107424,zavalse01,0
107425,zeuchtj01,0
107426,zimmebr01,0
107427,zimmejo02,0


# 2.2 

In [196]:
b2 = pd.read_sql_query(
"""
SELECT 
    HR,
    salary

FROM batting

INNER JOIN salaries ON
    batting.yearID = salaries.yearID

ORDER BY HR DESC

""", con=baseball)

b2

,HR,salary
0,73,4500000.0
1,73,270000.0
2,73,320000.0
3,73,200000.0
4,73,3450000.0
...,...,...
33740326,0,10400000.0
33740327,0,524000.0
33740328,0,524900.0
33740329,0,21733615.0


In [ ]:
"""
Based on the query, there doesnt seem to be a relation between homeruns
and salary.
"""